In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
from functools import reduce
from sklearn.metrics import mean_squared_error as mse
from tqdm.auto import tqdm
from tqdm import tqdm
from copy import deepcopy
import holidays
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor

plt.rcParams.update({'figure.max_open_warning': 0})

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 157)

In [2]:
df = pd.read_csv('Question_2_dataset.csv', index_col='localminute', parse_dates=True, date_parser=lambda col: pd.to_datetime(col, utc=True))
df

,dataid,meter_value,hours_passed,weekday,is_holiday,is_weekday,is_office_hour,is_sleeping,month,year,date,hour
localminute,,,,,,,,,,,,
2015-10-01 05:00:00+00:00,739.0,88858.000000,0,3,0,1,0,1,10,2015,1,5
2015-10-01 06:00:00+00:00,739.0,88858.000000,1,3,0,1,0,1,10,2015,1,6
2015-10-01 07:00:00+00:00,739.0,88860.000000,2,3,0,1,0,1,10,2015,1,7
2015-10-01 08:00:00+00:00,739.0,88860.000000,3,3,0,1,0,1,10,2015,1,8
2015-10-01 09:00:00+00:00,739.0,88860.000000,4,3,0,1,1,1,10,2015,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-03-08 12:00:00+00:00,4874.0,314452.311622,779,1,0,1,1,0,3,2016,8,12
2016-03-08 13:00:00+00:00,4874.0,314461.233716,780,1,0,1,1,0,3,2016,8,13
2016-03-08 14:00:00+00:00,4874.0,314470.155811,781,1,0,1,1,0,3,2016,8,14


In [3]:
unique_households = df['dataid'].unique()
unique_households.sort()
unique_households = np.delete(unique_households, [83,140])
unique_households

array([  35.,   44.,   77.,   94.,  114.,  187.,  222.,  252.,  370.,
        483.,  484.,  661.,  739.,  744.,  871., 1042., 1086., 1103.,
       1185., 1283., 1403., 1415., 1507., 1556., 1589., 1619., 1697.,
       1714., 1718., 1790., 1791., 1792., 1800., 1801., 2018., 2034.,
       2072., 2094., 2129., 2233., 2335., 2378., 2449., 2461., 2470.,
       2575., 2638., 2645., 2755., 2814., 2818., 2945., 2946., 2965.,
       2980., 3036., 3039., 3134., 3310., 3367., 3527., 3544., 3577.,
       3635., 3723., 3778., 3849., 3893., 3918., 4029., 4031., 4193.,
       4228., 4296., 4352., 4356., 4373., 4421., 4447., 4514., 4671.,
       4732., 4767., 4998., 5129., 5131., 5193., 5275., 5317., 5395.,
       5403., 5439., 5484., 5545., 5636., 5658., 5785., 5810., 5814.,
       5892., 5972., 6101., 6412., 6505., 6578., 6673., 6685., 6830.,
       6836., 6863., 6910., 7016., 7017., 7030., 7117., 7287., 7429.,
       7460., 7566., 7674., 7682., 7739., 7741., 7794., 7900., 7919.,
       7965., 7989.,

In [4]:
import random
import string

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

password_list = [get_random_string(8) for _ in unique_households]

In [32]:
df = pd.DataFrame({'dataid' : unique_households,
                    'password' : password_list})
df.to_csv('password_list.csv', index=False)

In [6]:
# df.to_csv('training.csv', index=False)
df = pd.read_csv('Question_2_dataset.csv', index_col='localminute', parse_dates=True, date_parser=lambda col: pd.to_datetime(col, utc=True))
df.reset_index(inplace=True)
df

,localminute,dataid,meter_value,hours_passed,weekday,is_holiday,is_weekday,is_office_hour,is_sleeping,month,year,date,hour
0,2015-10-01 05:00:00+00:00,739.0,88858.000000,0,3,0,1,0,1,10,2015,1,5
1,2015-10-01 06:00:00+00:00,739.0,88858.000000,1,3,0,1,0,1,10,2015,1,6
2,2015-10-01 07:00:00+00:00,739.0,88860.000000,2,3,0,1,0,1,10,2015,1,7
3,2015-10-01 08:00:00+00:00,739.0,88860.000000,3,3,0,1,0,1,10,2015,1,8
4,2015-10-01 09:00:00+00:00,739.0,88860.000000,4,3,0,1,1,1,10,2015,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610627,2016-03-08 12:00:00+00:00,4874.0,314452.311622,779,1,0,1,1,0,3,2016,8,12
610628,2016-03-08 13:00:00+00:00,4874.0,314461.233716,780,1,0,1,1,0,3,2016,8,13
610629,2016-03-08 14:00:00+00:00,4874.0,314470.155811,781,1,0,1,1,0,3,2016,8,14
610630,2016-03-08 15:00:00+00:00,4874.0,314479.077905,782,1,0,1,1,0,3,2016,8,15


In [7]:
us_holidays = holidays.UnitedStates()

def change_time(row):
    row['localminute'] = row['localminute']+relativedelta(months=5*12+9)
    return row

df = df.apply(lambda x : change_time(x), axis=1)
df

,localminute,dataid,meter_value,hours_passed,weekday,is_holiday,is_weekday,is_office_hour,is_sleeping,month,year,date,hour
0,2021-07-01 05:00:00+00:00,739.0,88858.000000,0,3,0,1,0,1,10,2015,1,5
1,2021-07-01 06:00:00+00:00,739.0,88858.000000,1,3,0,1,0,1,10,2015,1,6
2,2021-07-01 07:00:00+00:00,739.0,88860.000000,2,3,0,1,0,1,10,2015,1,7
3,2021-07-01 08:00:00+00:00,739.0,88860.000000,3,3,0,1,0,1,10,2015,1,8
4,2021-07-01 09:00:00+00:00,739.0,88860.000000,4,3,0,1,1,1,10,2015,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610627,2021-12-08 12:00:00+00:00,4874.0,314452.311622,779,1,0,1,1,0,3,2016,8,12
610628,2021-12-08 13:00:00+00:00,4874.0,314461.233716,780,1,0,1,1,0,3,2016,8,13
610629,2021-12-08 14:00:00+00:00,4874.0,314470.155811,781,1,0,1,1,0,3,2016,8,14
610630,2021-12-08 15:00:00+00:00,4874.0,314479.077905,782,1,0,1,1,0,3,2016,8,15


In [8]:
df.to_csv('Past_data.csv', index=False)

In [ ]:
features = ['is_holiday', 'is_weekday', 'hours_passed']
hours_to_predict = 120
def plot_overall_timeseries(house_df, train_df, test_df, house):
    mean_squared_error = mse(house_df['predicted_meter_value'], house_df['meter_value'])
    
    plt.figure(figsize=(12, 8))
    plt.scatter(train_df['localminute'], train_df['meter_value'], c='blue',  label = 'actual training value')
    plt.scatter(test_df['localminute'], test_df['meter_value'], c='red' ,  label = 'actual test value')
    plt.scatter(test_df['localminute'], test_df['predicted_meter_value'], c='orange' ,  label = 'predicted test value')
    plt.xlabel('Date')
    plt.ylabel('Gas Consumption')
    plt.title(f'Time Series Plot - Household {house} Overall MSE : {mean_squared_error}')
    plt.legend()
    return plt

def plot_testing_scatter(test_df, house):
    mean_squared_error = mse(test_df['predicted_meter_value'], test_df['meter_value'])
    plt.figure(figsize=(12, 8))
    plt.plot(test_df['meter_value'], test_df['predicted_meter_value'], 'o' ,c='red' ,label = 'scatter of actual vs predicted')
    plt.plot(test_df['meter_value'], test_df['meter_value'], c='green', label = 'line for goodness of fit', linewidth=3)
    plt.xlabel('Actual Meter Value')
    plt.ylabel('Predicted Meter Value')
    plt.title(f' Scatter Plot - Household {house} Testing MSE : {mean_squared_error}')
    plt.legend()
    return plt
    
def plot_lr_vs_svr_line(test_df, house):
    mean_lr_squared_error = mse(test_df['predicted_lr_meter_value'], test_df['meter_value'])
    mean_svr_squared_error = mse(test_df['predicted_svr_meter_value'], test_df['meter_value'])
    true_mean_squared_error = mse(test_df['mean_predicted_meter_value'], test_df['meter_value'])
    plt.figure(figsize=(12, 8))
    plt.plot(test_df['meter_value'], test_df['meter_value'], c='red' ,label = 'line for goodness of fit', linewidth=3)
    plt.plot(test_df['meter_value'], test_df['predicted_svr_meter_value'], '+' , c='orange' ,  label = 'predicted svr test value', linewidth=3)
    plt.plot(test_df['meter_value'], test_df['predicted_lr_meter_value'], '+' , c='green' ,  label = 'predicted lr test value', linewidth=3)
    plt.plot(test_df['meter_value'], test_df['mean_predicted_meter_value'], 'o' , c='black' ,  label = 'predicted mean test value', linewidth=3)
    plt.xlabel('Actual Meter Value')
    plt.ylabel('Predicted Meter Value')
    plt.title(f'Household {house} \nTesting SVR MSE : {mean_svr_squared_error}\nTesting LR MSE : {mean_lr_squared_error}\nEnsemble MSE : {true_mean_squared_error}')
    plt.legend()
    return plt

def train_each_house(model, hours_to_predict):
    def get_mean(row):
        return (row['predicted_svr_meter_value'] + row['predicted_lr_meter_value']) / 2
    
    for house in unique_households:
        house_df = df.loc[df['dataid'] == house]
        # we simply interpolate the hours with no data since meter_value is the cumulative usage.
        hourly_df_train = house_df[:-hours_to_predict]
        hourly_df_test = house_df[-hours_to_predict:]
        if model == "svr":
            temp_model = make_pipeline(StandardScaler(), SVR(kernel='linear', C=100, gamma=0.1, epsilon=.1))
            temp_model.fit(hourly_df_train[features], hourly_df_train['meter_value'])
        elif model == 'lr' :
            temp_model = make_pipeline(StandardScaler(), LinearRegression())
            temp_model.fit(hourly_df_train[features], hourly_df_train['meter_value'])
        else :
            svr_model = make_pipeline(StandardScaler(), SVR(kernel='linear', C=100, gamma=0.1, epsilon=.1))
            svr_model.fit(hourly_df_train[features], hourly_df_train['meter_value'])
            lr_model = make_pipeline(StandardScaler(), LinearRegression())
            lr_model.fit(hourly_df_train[features], hourly_df_train['meter_value'])
        if model == 'svr' or model == 'lr':
            hourly_df_train['predicted_meter_value'] = temp_model.predict(hourly_df_train[features])
            hourly_df_test['predicted_meter_value'] = temp_model.predict(hourly_df_test[features])
            house_df['predicted_meter_value'] = temp_model.predict(house_df[features])
            plot1 = plot_overall_timeseries(house_df, hourly_df_train, hourly_df_test, house)
            plot2 = plot_testing_scatter(hourly_df_test, house)
        else :
            hourly_df_test['predicted_svr_meter_value'] = svr_model.predict(hourly_df_test[features])
            hourly_df_test['predicted_lr_meter_value'] = lr_model.predict(hourly_df_test[features])
            hourly_df_test['mean_predicted_meter_value'] = hourly_df_test.apply(lambda row : get_mean(row), axis=1)
            plot = plot_lr_vs_svr_line(hourly_df_test, house)
        plt.show()
